In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional, Union
import scipy.stats as stats
from scipy.optimize import minimize_scalar
import warnings
warnings.filterwarnings('ignore')

### Real-data Algorithm

In [25]:
def calculate_reservation_prices(sigma: float, T: float, s: float, q: int, t: float, gamma: float) -> Tuple[float, float, float]:
    time_to_expiry = T - t
    r = s - q * gamma * sigma**2 * time_to_expiry
    r_bid = s + (-1 - 2*q) * gamma * sigma**2 * time_to_expiry / 2
    r_ask = s + (1 - 2*q) * gamma * sigma**2 * time_to_expiry / 2
    
    return r, r_bid, r_ask

In [27]:
def calculate_optimal_spread(T: float, gamma: float, t: float, k: float) -> float:
    time_to_expiry = T - t
    
    if time_to_expiry <= 0:
        return 0.0
    if gamma <= 0:
        return 0.0
        
    try:
        spread = gamma * sigma**2 * time_to_expiry + (2/gamma) * np.log(1 + gamma/k)
    except (OverflowError, ZeroDivisionError):
        spread = gamma * sigma**2 * time_to_expiry
    
    return max(spread, 0.01)

In [29]:
def calculate_order_intensity(A: float, k: float, delta: float) -> float:
    if delta <= 0:
        return A

    try:
        intensity = A * np.exp(-k * delta)
    except OverflowError:
        intensity = 0.0
    return max(intensity, 0.0)

In [5]:
def simulate_strategy(df: pd.DataFrame, strategy_type: str, gamma: float, T: float, dt: float, A: float, k: float, N: float) -> Dict:
    price_path = df['mid_price']
    sigma_path = df['sigma']
    inventory = 0
    cash = 0.0

    inventory_history = []
    cash_history = []
    spread_history = []
    n_steps = N
    for i in range(n_steps):
        t = i * dt
        s = price_path[i]
        sigma = sigma_path[i]
        r, r_bid, r_ask = calculate_reservation_prices(sigma, T, s, inventory, t, gamma)
        total_spread = calculate_optimal_spread(T, gamma, t, k)

        if strategy_type == 'inventory':  # Center around reservation price
                
            bid_price = r - total_spread / 2
            ask_price = r + total_spread / 2
                
        elif strategy_type == 'symmetric':  # Center around mid-price
                
            bid_price = s - total_spread / 2
            ask_price = s + total_spread / 2
        else:
            raise ValueError(f"Unknown strategy type: {strategy_type}")

        delta_bid = s - bid_price
        delta_ask = ask_price - s

        # Ensure positive spreads
        delta_bid = max(delta_bid, 0.01)
        delta_ask = max(delta_ask, 0.01)

        lambda_bid = calculate_order_intensity(A, k, delta_bid)
        lambda_ask = calculate_order_intensity(A, k, delta_ask)

        # Simulate order arrivals (Poisson process)
        prob_bid = min(lambda_bid * dt, 1.0)
        prob_ask = min(lambda_ask * dt, 1.0)

        bid_order = np.random.random() < prob_bid
        ask_order = np.random.random() < prob_ask

        if bid_order:
            inventory += 1
            cash -= bid_price

        if ask_order:
            inventory -= 1
            cash += ask_price
                
        inventory_history.append(inventory)
        cash_history.append(cash)
        spread_history.append(delta_bid + delta_ask)

    # Final P&L calculation
    final_price = price_path[n-1]
    final_pnl = cash + inventory * final_price

    return {'final_pnl': final_pnl, 'final_inventory': inventory, 'final_cash': cash, 'final_price': final_price,
            'inventory_history': inventory_history,
            'cash_history': cash_history,
            'spread_history': spread_history,
            'mean_spread': np.mean(spread_history),
            'inventory_std': np.std(inventory_history)
        }

## Launching the algorithm

In [9]:
# data = pd.DataFrame({'mid_price': ..., 'sigma': ...})

### Gamma = 0.01

In [11]:
# result_inventory_0_01 = simulate_strategy(data, 'inventory', 0.01, T, t, A, k, N)
# result_symmetric_0_01 = simulate_strategy(data, 'symmetric', 0.01, T, t, A, k, N)

### Gamma = 0.05

In [21]:
# result_inventory_0_05 = simulate_strategy(data, 'inventory', 0.05, T, t, A, k, N)
# result_symmetric_0_05 = simulate_strategy(data, 'symmetric', 0.05, T, t, A, k, N)

### Gamma = 0.1

In [17]:
# result_inventory_0_1 = simulate_strategy(data, 'inventory', 0.1, T, t, A, k, N)
# result_symmetric_0_1 = simulate_strategy(data, 'symmetric', 0.1, T, t, A, k, N)